In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler, LabelEncoder
from sklearn.compose import make_column_transformer
from sklearn.multiclass import OneVsOneClassifier, OneVsRestClassifier
from lightgbm.sklearn import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, StackingClassifier
from sklearn.svm import SVC
from catboost import CatBoostClassifier
from sklearn.feature_selection import SelectFromModel

In [2]:
trade_area = pd.read_csv("../../data/Smoothie King/smoothie_king_trade_area_variables.csv")
poi = pd.read_csv('../../data/Smoothie King/processed_poi.csv')
stores = pd.read_csv('../../data/Smoothie King/smoothie_king_stores.csv')
demographic = pd.read_csv("../../data/Smoothie King/processed_demographic.csv")
trade_area = pd.read_csv("../../data/Smoothie King/processed_trade_area.csv")
# to_remove = [col for col in trade_area.columns.tolist() if "_p_" in col]
# trade_area = trade_area.drop(columns=to_remove)
# merged = stores.merge(trade_area, left_on="store", right_on="store_num").merge(poi, on="store").merge(demographic, on="store")
merged = stores.merge(trade_area, left_on="store", right_on="store_num").merge(poi)
merged = merged.drop(columns=["store_num", "country_code"])
merged

,store,longitude,latitude,category,cbsa_name,dma_name,state_name,market_size,store_density,age0018_p_ta,...,osm_highway_exits_count_3mi,osm_highway_exits_count_5mi,osm_nearest_exit_dist,places_of_worship_10mi,places_of_worship_1mi,places_of_worship_2mi,places_of_worship_3mi,places_of_worship_5mi,transitstop_nearest_dist,transitstops
0,SK 1504,-97.650392,30.519353,SHOPPING,"Austin-Round Rock, TX","Austin, TX",Texas,Large Metro (2),Light Suburban,0.2907,...,17,45,1.875541,314,5,30,76,128,14.818824,0
1,SK 0057,-88.171150,30.672501,SHOPPING,"Mobile, AL","Mobile et al, AL-FL",Alabama,Medium City (4),Light Suburban,0.2416,...,3,14,2.617072,305,12,82,109,154,49.572856,0
2,SK 1415,-90.535722,38.784250,HOME,"St. Louis, MO-IL","St. Louis, MO",Missouri,Very Large Metro (1),Light Suburban,0.2198,...,25,45,0.194937,277,6,23,57,78,3.518308,0
3,SK 1231,-80.134700,26.100737,TRAVEL,"Miami-Fort Lauderdale et al, FL","Miami-Ft. Lauderdale, FL",Florida,Very Large Metro (1),Suburban,0.2060,...,31,61,1.393043,1202,10,50,116,393,0.021790,25
4,SK 1535,-96.856651,32.996408,WORK,"Dallas-Fort Worth-Arlington, TX","Dallas-Ft. Worth, TX",Texas,Very Large Metro (1),Light Suburban,0.2866,...,23,72,0.711949,903,12,58,117,273,0.055289,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
791,SK 1536,-96.872596,32.647809,HOME,"Dallas-Fort Worth-Arlington, TX","Dallas-Ft. Worth, TX",Texas,Very Large Metro (1),Exurban,0.2745,...,36,81,0.261721,866,8,40,115,323,0.012345,24
792,SK 1886,-105.077634,40.564695,OTHER,"Fort Collins, CO","Denver, CO",Colorado,Medium City (4),Light Suburban,0.2083,...,0,10,4.012518,175,20,71,112,128,0.189059,13
793,SK 0162,-95.478001,30.316531,SHOPPING,"Houston-The Woodlands et al, TX","Houston, TX",Texas,Very Large Metro (1),Exurban,0.2730,...,15,23,0.397305,140,20,42,53,68,0.699036,0
794,SK 1449,-78.968258,35.064994,SHOPPING,"Fayetteville, NC","Raleigh et al, NC",North Carolina,Medium City (4),Exurban,0.2624,...,12,46,0.706073,293,8,43,85,169,4.771075,0


In [3]:
merged = merged.drop(columns=["store", "longitude", "latitude", "state_name", "cbsa_name", "dma_name"])
merged = merged.dropna()

In [4]:
le = LabelEncoder()
merged["category"] = le.fit_transform(merged["category"])
merged

,category,market_size,store_density,age0018_p_ta,age65pl_p_ta,age85pl_p_ta,asian_p_ta,avg_faminc_ta,avghhinc_ta,black_p_ta,...,osm_highway_exits_count_3mi,osm_highway_exits_count_5mi,osm_nearest_exit_dist,places_of_worship_10mi,places_of_worship_1mi,places_of_worship_2mi,places_of_worship_3mi,places_of_worship_5mi,transitstop_nearest_dist,transitstops
0,2,Large Metro (2),Light Suburban,0.2907,0.0909,0.0050,0.0555,108837.0,106093.0,0.0985,...,17,45,1.875541,314,5,30,76,128,14.818824,0
1,2,Medium City (4),Light Suburban,0.2416,0.1537,0.0134,0.0327,80752.0,70536.0,0.3890,...,3,14,2.617072,305,12,82,109,154,49.572856,0
2,0,Very Large Metro (1),Light Suburban,0.2198,0.1926,0.0147,0.0235,106893.0,97112.0,0.0552,...,25,45,0.194937,277,6,23,57,78,3.518308,0
3,3,Very Large Metro (1),Suburban,0.2060,0.2108,0.0185,0.0203,134373.0,149467.0,0.0998,...,31,61,1.393043,1202,10,50,116,393,0.021790,25
4,4,Very Large Metro (1),Light Suburban,0.2866,0.1283,0.0057,0.0719,102367.0,96066.0,0.1935,...,23,72,0.711949,903,12,58,117,273,0.055289,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
791,0,Very Large Metro (1),Exurban,0.2745,0.1529,0.0111,0.0085,66363.0,59465.0,0.5114,...,36,81,0.261721,866,8,40,115,323,0.012345,24
792,1,Medium City (4),Light Suburban,0.2083,0.1078,0.0100,0.0304,95226.0,73773.0,0.0159,...,0,10,4.012518,175,20,71,112,128,0.189059,13
793,2,Very Large Metro (1),Exurban,0.2730,0.1367,0.0111,0.0172,89271.0,86552.0,0.1078,...,15,23,0.397305,140,20,42,53,68,0.699036,0
794,2,Medium City (4),Exurban,0.2624,0.1357,0.0111,0.0338,67117.0,62256.0,0.3939,...,12,46,0.706073,293,8,43,85,169,4.771075,0


In [5]:
le.classes_

array(['HOME', 'OTHER', 'SHOPPING', 'TRAVEL', 'WORK'], dtype=object)

In [6]:
train_df, test_df = train_test_split(merged, test_size=0.1, random_state=42)
X_train = train_df.drop(columns=["category"])
y_train = train_df["category"]
X_test = test_df.drop(columns=["category"])
y_test = test_df["category"]

In [7]:
X_train

,market_size,store_density,age0018_p_ta,age65pl_p_ta,age85pl_p_ta,asian_p_ta,avg_faminc_ta,avghhinc_ta,black_p_ta,boomer_p_ta,...,osm_highway_exits_count_3mi,osm_highway_exits_count_5mi,osm_nearest_exit_dist,places_of_worship_10mi,places_of_worship_1mi,places_of_worship_2mi,places_of_worship_3mi,places_of_worship_5mi,transitstop_nearest_dist,transitstops
347,Very Large Metro (1),Exurban,0.2969,0.1014,0.0071,0.0261,93118.0,89967.0,0.6010,0.1739,...,10,33,2.615653,746,4,13,53,162,1.748239,0
111,Very Large Metro (1),Exurban,0.2584,0.1199,0.0059,0.0633,72243.0,67727.0,0.5925,0.2033,...,14,21,0.388134,472,7,38,63,149,1.455125,0
54,Small Town (6),Exurban,0.2161,0.1461,0.0131,0.0151,73789.0,81607.0,0.4227,0.1953,...,0,0,7.056117,37,14,23,27,30,31.770532,0
747,Large City (3),Exurban,0.2736,0.1147,0.0083,0.0446,135460.0,134123.0,0.0187,0.1858,...,6,27,1.977634,153,4,14,23,48,1.319867,0
194,Very Large Metro (1),Light Suburban,0.2066,0.2145,0.0299,0.0209,104670.0,100387.0,0.0129,0.2525,...,16,46,1.326653,623,9,42,91,237,0.025833,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,Very Large Metro (1),Exurban,0.2466,0.1358,0.0071,0.1914,137597.0,150433.0,0.1700,0.2253,...,14,29,0.203553,477,7,15,26,102,9.251025,0
107,Very Large Metro (1),Suburban,0.1881,0.1878,0.0199,0.0884,146098.0,161364.0,0.0730,0.2473,...,39,101,0.331632,1753,22,77,176,410,0.040567,11
275,Small City (5),Exurban,0.2144,0.1539,0.0150,0.0187,75062.0,62347.0,0.0340,0.1862,...,6,10,0.200039,48,5,15,26,40,44.211875,0
443,Very Large Metro (1),Exurban,0.2626,0.1129,0.0051,0.1057,103601.0,92428.0,0.4126,0.1956,...,21,58,1.637729,511,8,32,55,130,0.057342,11


In [8]:
ordinal_features = ["market_size", "store_density"]
numeric_features = X_train.select_dtypes(include=[np.number]).columns.tolist()

In [9]:
market_levels = [
    "Small Town (6)",
    "Small City (5)",
    "Medium City (4)",
    "Large City (3)",
    "Large Metro (2)",
    "Very Large Metro (1)"
]
density_levels = [
    "Rural",
    "Exurban",
    "Suburban",
    "Light Suburban",
    "Light Urban",
    "Urban",
    "Super Urban"
]

In [10]:
ordinal_transformer = OrdinalEncoder(categories=[market_levels, density_levels], dtype=int)

preprocessor = make_column_transformer(
    (make_pipeline(SimpleImputer(strategy="constant", fill_value=0), StandardScaler()), numeric_features),
    (ordinal_transformer, ordinal_features),
)

In [184]:
class_weight = {
    "HOME": 0.24,
    "OTHER": 0.16,
    "SHOPPING": 0.22,
    "TRAVEL": 0.15,
    "WORK": 0.23
}
encoded_class_weights = {i: class_weight[label] for i, label in enumerate(le.classes_)}
class_weight = encoded_class_weights

## Build Logistic Regression Models

In [165]:
pipe_lr = make_pipeline(
    preprocessor,
    LogisticRegression(C=0.15, penalty="l2", random_state=42, solver="saga", max_iter=10000, multi_class="ovr", n_jobs=-1, class_weight=class_weight)
)
pipe_lr.fit(X_train, y_train);

In [166]:
pipe_lr.score(X_train, y_train)

0.6577086280056577

In [167]:
pipe_lr.score(X_test, y_test)

0.6329113924050633

In [168]:
pipe_lr_multi = make_pipeline(
    preprocessor,
    LogisticRegression(C=0.1, penalty="l2", random_state=42, solver="saga", max_iter=10000, multi_class="multinomial", n_jobs=-1, 
                       class_weight=class_weight)
)
pipe_lr_multi.fit(X_train, y_train);

In [169]:
pipe_lr_multi.score(X_train, y_train)

0.6661951909476662

In [170]:
pipe_lr_multi.score(X_test, y_test)

0.6075949367088608

In [175]:
prediction_result = pd.DataFrame({
    "True label": y_test,
    "pipe_lr": pipe_lr.predict(X_test),
    "pipe_lr_multi": pipe_lr_multi.predict(X_test)
})

In [319]:
rf_tuned = make_pipeline(
    preprocessor,
    RandomForestClassifier(n_estimators=100, max_depth=30, max_leaf_nodes=30, class_weight=class_weight, 
                           min_samples_leaf=10, min_samples_split=30, n_jobs=-1, random_state=42)
)
rf_tuned.fit(X_train, y_train);

In [320]:
rf_tuned.score(X_train, y_train)

0.6888260254596889

In [321]:
rf_tuned.score(X_test, y_test)

0.6708860759493671

In [22]:
# Takes long time to run

# rf_ovr_test = make_pipeline(
#     preprocessor,
#     SelectFromModel(
#         LogisticRegression(C=0.15, penalty="l2", random_state=42, solver="saga", max_iter=10000, multi_class="ovr", 
#                            n_jobs=-1, class_weight=class_weight)
#     ),
#     RandomForestClassifier(n_estimators=100, max_depth=30, max_leaf_nodes=30, class_weight="balanced", 
#                            min_samples_leaf=10, min_samples_split=30, n_jobs=-1, random_state=42)
# )
# rf_param_grid = {
#     "randomforestclassifier__n_estimators": [25, 50, 100, 150],
#     # "randomforestclassifier__max_features": ["sqrt", "log2", None],
#     "randomforestclassifier__max_features": [5, 10, 15, 20, 30],
#     "randomforestclassifier__max_depth": [5, 10, 20, 30],
#     "randomforestclassifier__max_leaf_nodes": [30, 50, 70],
#     "randomforestclassifier__min_samples_leaf": [10, 20, 30, 40, 50],
#     "randomforestclassifier__min_samples_split": [10, 20, 30, 40],
#     "randomforestclassifier__class_weight": [None, "balanced", class_weight]
# }
# rf_ovr_search = RandomizedSearchCV(
#     rf_ovr_test, rf_param_grid, n_iter=30, cv=5, n_jobs=-1, random_state=42
# )

In [23]:
# rf_ovr_search.fit(X_train, y_train);

In [24]:
# pd.DataFrame(rf_ovr_search.cv_results_)[
#     [
#         "param_randomforestclassifier__n_estimators",
#         "param_randomforestclassifier__max_features",
#         "param_randomforestclassifier__max_depth",
#         "param_randomforestclassifier__max_leaf_nodes",
#         "param_randomforestclassifier__min_samples_leaf",
#         "param_randomforestclassifier__min_samples_split",
#         "param_randomforestclassifier__class_weight",
#         "mean_fit_time",
#         "rank_test_score",
#         "mean_test_score",
#     ]
# ].set_index("rank_test_score").sort_values(by="mean_test_score", ascending=False)

In [ ]:
# rf_ovr_search.best_params_

In [ ]:
# rf_ovr_search.score(X_train, y_train)

In [ ]:
# rf_ovr_search.score(X_test, y_test)

In [171]:
pipe_lr_rf = make_pipeline(
    preprocessor,
    SelectFromModel(
        LogisticRegression(C=0.15, penalty="l2", random_state=42, solver="saga", max_iter=10000, multi_class="ovr", 
                           n_jobs=-1, class_weight=class_weight)
    ),
    RandomForestClassifier(n_estimators=100, max_depth=20, max_leaf_nodes=70, class_weight=class_weight, 
                           min_samples_leaf=10, min_samples_split=30, n_jobs=-1, random_state=42)
)
pipe_lr_rf.fit(X_train, y_train);

In [172]:
pipe_lr_rf.score(X_train, y_train)

0.6944837340876945

In [173]:
pipe_lr_rf.score(X_test, y_test)

0.6582278481012658

In [346]:
pipe_lr_rf_test = make_pipeline(
    preprocessor,
    SelectFromModel(
        LogisticRegression(C=0.15, penalty="l2", random_state=42, solver="saga", max_iter=10000, multi_class="ovr", 
                           n_jobs=-1, class_weight=class_weight)
    ),
    RandomForestClassifier(n_estimators=100, max_depth=20, max_leaf_nodes=70, class_weight=class_weight, 
                           min_samples_leaf=10, min_samples_split=30, n_jobs=-1, random_state=42)
)

In [348]:
lr_rf_param = {
    "selectfrommodel__estimator__C": [0.1, 0.15, 0.2, 0.3, 0.4],
    "selectfrommodel__estimator__penalty": ["l1", "l2"],
    "selectfrommodel__estimator__multi_class": ["ovr", "multinomial"],
    "selectfrommodel__estimator__class_weight": [None, "balanced", class_weight]
}
lr_rf_search = RandomizedSearchCV(
    pipe_lr_rf_test, lr_rf_param, n_iter=30, cv=5, n_jobs=-1, random_state=42
)
lr_rf_search.fit(X_train, y_train);

In [369]:
pd.DataFrame(lr_rf_search.cv_results_)[
    [
        "param_selectfrommodel__estimator__C",
        "param_selectfrommodel__estimator__penalty",
        "param_selectfrommodel__estimator__multi_class",
        "param_selectfrommodel__estimator__class_weight",
        "mean_fit_time",
        "rank_test_score",
        "mean_test_score",
    ]
].set_index("rank_test_score").sort_values(by="mean_test_score", ascending=False).head(10)

,param_selectfrommodel__estimator__C,param_selectfrommodel__estimator__penalty,param_selectfrommodel__estimator__multi_class,param_selectfrommodel__estimator__class_weight,mean_fit_time,mean_test_score
rank_test_score,,,,,,
1,0.1,l1,ovr,None,23.700607,0.451184
2,0.4,l1,multinomial,"{0: 0.24, 1: 0.16, 2: 0.22, 3: 0.15, 4: 0.23}",30.404138,0.446898
3,0.1,l2,ovr,balanced,23.567962,0.445550
4,0.2,l1,multinomial,"{0: 0.24, 1: 0.16, 2: 0.22, 3: 0.15, 4: 0.23}",37.576730,0.445520
5,0.3,l2,ovr,"{0: 0.24, 1: 0.16, 2: 0.22, 3: 0.15, 4: 0.23}",26.829270,0.444141
6,0.15,l2,ovr,None,22.659697,0.442723
6,0.1,l1,multinomial,balanced,20.993425,0.442723
8,0.4,l2,ovr,"{0: 0.24, 1: 0.16, 2: 0.22, 3: 0.15, 4: 0.23}",15.836400,0.441325
9,0.2,l1,multinomial,balanced,19.372758,0.441285


In [350]:
lr_rf_search.score(X_train, y_train)

0.6817538896746818

In [351]:
lr_rf_search.score(X_test, y_test)

0.6708860759493671

In [353]:
lr_rf_l1 = lr_rf_search.best_estimator_

In [354]:
prediction_result["lr_rf_l1"] = lr_rf_l1.predict(X_test)

In [367]:
pipe_lr_rf_rank2 = make_pipeline(
    preprocessor,
    SelectFromModel(
        LogisticRegression(C=0.4, penalty="l1", random_state=42, solver="saga", max_iter=10000, multi_class="multinomial", 
                           n_jobs=-1, class_weight=class_weight)
    ),
    RandomForestClassifier(n_estimators=100, max_depth=20, max_leaf_nodes=70, class_weight=class_weight, 
                           min_samples_leaf=10, min_samples_split=30, n_jobs=-1, random_state=42)
)
pipe_lr_rf_rank2.fit(X_train, y_train);

In [368]:
print(pipe_lr_rf_rank2.score(X_train, y_train))
print(pipe_lr_rf_rank2.score(X_test, y_test))

0.669024045261669
0.6582278481012658


In [322]:
prediction_result["rf"] = rf_tuned.predict(X_test)
prediction_result["lr_rf"] = pipe_lr_rf.predict(X_test)

In [370]:
lr_rf_ovr = make_pipeline(
    preprocessor,
    SelectFromModel(
        LogisticRegression(C=0.15, penalty="l2", random_state=42, solver="saga", max_iter=10000, multi_class="ovr", 
                           n_jobs=-1, class_weight=class_weight)
    ),
    OneVsRestClassifier(
        RandomForestClassifier(n_estimators=100, max_depth=30, max_leaf_nodes=30, class_weight=None, 
                               min_samples_leaf=10, min_samples_split=30, n_jobs=-1, random_state=42)
    )
)
lr_rf_param = {
    "selectfrommodel__estimator__C": [0.1, 0.15, 0.2, 0.3, 0.4],
    "selectfrommodel__estimator__penalty": ["l1", "l2"],
    "selectfrommodel__estimator__multi_class": ["ovr", "multinomial"],
    "selectfrommodel__estimator__class_weight": [None, "balanced", class_weight]
}

In [371]:
lr_rf_ovr_search = RandomizedSearchCV(
    lr_rf_ovr, lr_rf_param, n_iter=30, cv=5, n_jobs=-1, random_state=42
)
lr_rf_ovr_search.fit(X_train, y_train);

RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('columntransformer',
                                              ColumnTransformer(transformers=[('pipeline',
                                                                               Pipeline(steps=[('simpleimputer',
                                                                                                SimpleImputer(fill_value=0,
                                                                                                              strategy='constant')),
                                                                                               ('standardscaler',
                                                                                                StandardScaler())]),
                                                                               ['age0018_p_ta',
                                                                                'age65pl_p_ta',
                        

In [372]:
pd.DataFrame(lr_rf_ovr_search.cv_results_)[
    [
        "param_selectfrommodel__estimator__C",
        "param_selectfrommodel__estimator__penalty",
        "param_selectfrommodel__estimator__multi_class",
        "param_selectfrommodel__estimator__class_weight",
        "mean_fit_time",
        "rank_test_score",
        "mean_test_score",
    ]
].set_index("rank_test_score").sort_values(by="mean_test_score", ascending=False).head(10)

,param_selectfrommodel__estimator__C,param_selectfrommodel__estimator__penalty,param_selectfrommodel__estimator__multi_class,param_selectfrommodel__estimator__class_weight,mean_fit_time,mean_test_score
rank_test_score,,,,,,
1,0.4,l1,multinomial,"{0: 0.24, 1: 0.16, 2: 0.22, 3: 0.15, 4: 0.23}",30.114802,0.466717
2,0.4,l2,ovr,"{0: 0.24, 1: 0.16, 2: 0.22, 3: 0.15, 4: 0.23}",19.623983,0.465328
3,0.1,l2,ovr,"{0: 0.24, 1: 0.16, 2: 0.22, 3: 0.15, 4: 0.23}",25.220479,0.463840
4,0.1,l2,multinomial,None,18.909653,0.462501
5,0.3,l1,multinomial,"{0: 0.24, 1: 0.16, 2: 0.22, 3: 0.15, 4: 0.23}",25.040832,0.461073
6,0.4,l1,ovr,"{0: 0.24, 1: 0.16, 2: 0.22, 3: 0.15, 4: 0.23}",27.719100,0.461053
7,0.2,l2,ovr,None,19.201422,0.459664
7,0.15,l1,ovr,None,20.590158,0.459664
9,0.1,l1,ovr,None,19.798306,0.459654


In [373]:
lr_rf_ovr_search.score(X_train, y_train)

0.7736916548797736

In [374]:
lr_rf_ovr_search.score(X_test, y_test)

0.6455696202531646

In [394]:
lr_rf_ovr = make_pipeline(
    preprocessor,
    SelectFromModel(
        LogisticRegression(C=0.15, penalty="l1", random_state=42, solver="saga", max_iter=10000, multi_class="multinomial", 
                           n_jobs=-1, class_weight=None)
    ),
    OneVsRestClassifier(
        RandomForestClassifier(n_estimators=100, max_depth=30, max_leaf_nodes=30, class_weight=None, 
                               min_samples_leaf=10, min_samples_split=30, n_jobs=-1, random_state=42)
    )
)
lr_rf_ovr.fit(X_train, y_train);

In [395]:
print(lr_rf_ovr.score(X_train, y_train))
print(lr_rf_ovr.score(X_test, y_test))

0.7835926449787836
0.6708860759493671


In [396]:
prediction_result["lr_rf_ovr"] = lr_rf_ovr.predict(X_test)

In [29]:
rf_ovr = make_pipeline(
    preprocessor,
    OneVsRestClassifier(
        RandomForestClassifier(n_estimators=100, max_depth=30, max_leaf_nodes=30, class_weight="balanced", 
                           min_samples_leaf=10, min_samples_split=30, n_jobs=-1, random_state=42)
    )
)
rf_ovr.fit(X_train, y_train);

In [30]:
rf_ovr.score(X_train, y_train)

0.8642149929278642

In [31]:
rf_ovr.score(X_test, y_test)

0.6582278481012658

In [306]:
lr_rf_ovr = make_pipeline(
    preprocessor,
    SelectFromModel(
        LogisticRegression(C=0.15, penalty="l2", random_state=42, solver="saga", max_iter=10000, multi_class="ovr", 
                           n_jobs=-1, class_weight=class_weight)
    ),
    OneVsRestClassifier(
        RandomForestClassifier(n_estimators=100, max_depth=30, max_leaf_nodes=30, class_weight=None, 
                               min_samples_leaf=10, min_samples_split=30, n_jobs=-1, random_state=42)
    )
)
lr_rf_ovr.fit(X_train, y_train);

In [307]:
lr_rf_ovr.score(X_train, y_train)

0.7906647807637907

In [308]:
lr_rf_ovr.score(X_test, y_test)

0.6582278481012658

In [309]:
prediction_result["lr_rf_ovr"] = lr_rf_ovr.predict(X_test)

In [32]:
prediction_result["rf_ovr"] = rf_ovr.predict(X_test)

In [187]:
pipe_lgbm_ovr = make_pipeline(
    preprocessor,
    OneVsRestClassifier(
        LGBMClassifier(random_state=42, n_jobs=-1)
    )
)
param_lgbm = {
    "onevsrestclassifier__estimator__n_estimators": [50, 100, 150, 200],
    "onevsrestclassifier__estimator__boosting_type": ["gbdt", "dart"],
    "onevsrestclassifier__estimator__learning_rate": [0.1, 0.3, 0.5, 0.7],
    "onevsrestclassifier__estimator__max_depth": [5, 10, 15, 20],
    "onevsrestclassifier__estimator__reg_alpha": [0.1, 0.3, 0.5, 0.7],
    "onevsrestclassifier__estimator__reg_lambda": [0.1, 0.3, 0.5, 0.7],
    # "onevsrestclassifier__estimator__min_child_samples": [20, 30, 40, 50],
    "onevsrestclassifier__estimator__num_leaves": [5, 10, 15, 20],
    "onevsrestclassifier__estimator__min_data_in_leaf": [30, 40, 50, 60],
    "onevsrestclassifier__estimator__class_weight": [None, "balanced"]
}
lgbm_search = RandomizedSearchCV(
    pipe_lgbm_ovr, param_lgbm, n_iter=30, cv=5, n_jobs=-1, random_state=42
)

In [188]:
lgbm_search.fit(X_train, y_train);

[LightGBM] [Warning] min_data_in_leaf is set=60, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=60


In [189]:
pd.DataFrame(lgbm_search.cv_results_)[
    [
        "param_onevsrestclassifier__estimator__n_estimators",
        "param_onevsrestclassifier__estimator__boosting_type",
        "param_onevsrestclassifier__estimator__learning_rate",
        "param_onevsrestclassifier__estimator__max_depth",
        "param_onevsrestclassifier__estimator__reg_alpha",
        "param_onevsrestclassifier__estimator__reg_lambda",
        "param_onevsrestclassifier__estimator__class_weight",
        # "param_onevsrestclassifier__estimator__min_child_samples",
        "param_onevsrestclassifier__estimator__num_leaves",
        "param_onevsrestclassifier__estimator__min_data_in_leaf",
        "mean_fit_time",
        "rank_test_score",
        "mean_test_score",
    ]
].set_index("rank_test_score").sort_index().head(20)

,param_onevsrestclassifier__estimator__n_estimators,param_onevsrestclassifier__estimator__boosting_type,param_onevsrestclassifier__estimator__learning_rate,param_onevsrestclassifier__estimator__max_depth,param_onevsrestclassifier__estimator__reg_alpha,param_onevsrestclassifier__estimator__reg_lambda,param_onevsrestclassifier__estimator__class_weight,param_onevsrestclassifier__estimator__num_leaves,param_onevsrestclassifier__estimator__min_data_in_leaf,mean_fit_time,mean_test_score
rank_test_score,,,,,,,,,,,
1,100,gbdt,0.1,5,0.7,0.1,None,10,60,5.750164,0.468135
2,150,gbdt,0.1,10,0.3,0.3,None,10,50,9.426673,0.466737
3,150,gbdt,0.3,10,0.7,0.7,None,20,30,4.585463,0.466717
4,150,gbdt,0.1,15,0.1,0.3,None,5,40,6.323008,0.462461
5,50,gbdt,0.1,5,0.1,0.3,None,5,60,2.577547,0.461083
6,50,dart,0.3,15,0.3,0.3,balanced,15,60,3.291978,0.461063
7,100,dart,0.3,20,0.3,0.5,balanced,15,30,9.460004,0.458246
8,100,dart,0.3,10,0.3,0.5,balanced,15,50,6.855371,0.458246
9,150,dart,0.3,5,0.5,0.7,None,20,60,4.676845,0.454031


In [190]:
lgbm_search.score(X_train, y_train)

1.0

In [191]:
lgbm_search.score(X_test, y_test)

0.5949367088607594

In [195]:
lgbm_test = make_pipeline(
    preprocessor,
    OneVsRestClassifier(
        LGBMClassifier(random_state=42, n_jobs=-1, n_estimators=150, boosting_type="gbdt", learning_rate=0.1, max_depth=10,
                      reg_alpha=0.7, reg_lambda=0.7, num_leaves=20, min_data_in_leaf=30, class_weight=None)
    )
)
lgbm_test.fit(X_train, y_train);

[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30


In [196]:
lgbm_test.score(X_train, y_train)

1.0

In [197]:
lgbm_test.score(X_test, y_test)

0.5443037974683544

In [185]:
lgbm_search.best_params_
# score=0.62025

{'onevsrestclassifier__estimator__reg_lambda': 0.3,
 'onevsrestclassifier__estimator__reg_alpha': 0.1,
 'onevsrestclassifier__estimator__num_leaves': 20,
 'onevsrestclassifier__estimator__n_estimators': 25,
 'onevsrestclassifier__estimator__min_data_in_leaf': 40,
 'onevsrestclassifier__estimator__min_child_samples': 20,
 'onevsrestclassifier__estimator__max_depth': 5,
 'onevsrestclassifier__estimator__learning_rate': 0.1,
 'onevsrestclassifier__estimator__boosting_type': 'gbdt'}

In [275]:
pipe_xgb = make_pipeline(
    preprocessor,
    OneVsRestClassifier(
        XGBClassifier(random_state=42, verbosity=0)
    )
)
param_xgb = {
    "onevsrestclassifier__estimator__booster": ["gbtree", "dart"],
    "onevsrestclassifier__estimator__learning_rate": [0.1, 0.3, 0.5, 0.7],
    "onevsrestclassifier__estimator__max_depth": [3, 5, 7, 9],
    "onevsrestclassifier__estimator__reg_alpha": [0.1, 0.3, 0.5, 0.7],
    "onevsrestclassifier__estimator__reg_lambda": [0.1, 0.3, 0.5, 0.7],
    "onevsrestclassifier__estimator__subsample": [0.3, 0.5, 0.7, 1],
    "onevsrestclassifier__estimator__min_child_weight": [1, 4, 7, 10]
}
xgb_search = RandomizedSearchCV(
    pipe_xgb, param_xgb, n_iter=30, cv=5, n_jobs=-1, random_state=42
)

In [276]:
xgb_search.fit(X_train, y_train);

c:\Users\ROG ZEPHYRUS\miniconda3\envs\capstone2023-env\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


In [278]:
pd.DataFrame(xgb_search.cv_results_)[
    [
        "param_onevsrestclassifier__estimator__booster",
        "param_onevsrestclassifier__estimator__learning_rate",
        "param_onevsrestclassifier__estimator__max_depth",
        "param_onevsrestclassifier__estimator__reg_alpha",
        "param_onevsrestclassifier__estimator__reg_lambda",
        "param_onevsrestclassifier__estimator__subsample",
        "param_onevsrestclassifier__estimator__min_child_weight",
        "mean_fit_time",
        "rank_test_score",
        "mean_test_score",
    ]
].set_index("rank_test_score").sort_index().head(20)

,param_onevsrestclassifier__estimator__booster,param_onevsrestclassifier__estimator__learning_rate,param_onevsrestclassifier__estimator__max_depth,param_onevsrestclassifier__estimator__reg_alpha,param_onevsrestclassifier__estimator__reg_lambda,param_onevsrestclassifier__estimator__subsample,param_onevsrestclassifier__estimator__min_child_weight,mean_fit_time,mean_test_score
rank_test_score,,,,,,,,,
1,dart,0.1,9,0.7,0.5,1,7,40.230910,0.463920
2,dart,0.1,5,0.1,0.5,0.7,4,56.979924,0.461043
3,dart,0.3,3,0.7,0.7,1,7,37.656043,0.453991
4,dart,0.3,5,0.1,0.7,0.7,1,57.571043,0.451234
5,dart,0.3,7,0.5,0.3,0.7,4,52.321766,0.448367
6,gbtree,0.1,3,0.1,0.1,0.7,7,6.498798,0.448347
7,gbtree,0.1,5,0.3,0.1,0.7,10,6.056025,0.448307
8,gbtree,0.1,9,0.1,0.1,0.5,1,12.621878,0.446878
9,gbtree,0.5,5,0.3,0.3,1,4,6.139763,0.445540


In [279]:
xgb_search.score(X_train, y_train)

0.9971711456859972

In [280]:
xgb_search.score(X_test, y_test)

0.5569620253164557

In [290]:
xgb_test = make_pipeline(
    preprocessor,
    OneVsRestClassifier(
        XGBClassifier(random_state=42, verbosity=0, booster="gbtree", learning_rate=0.1, max_depth=5, reg_alpha=0.5, reg_lambda=0.1,
                     subsample=0.7, min_child_weight=1)
    )
)
xgb_test.fit(X_train, y_train);

c:\Users\ROG ZEPHYRUS\miniconda3\envs\capstone2023-env\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


In [291]:
xgb_test.score(X_train, y_train)

1.0

In [292]:
xgb_test.score(X_test, y_test)
# XGBClassifier(random_state=42, verbosity=0, booster="gbtree", learning_rate=0.1, max_depth=5, reg_alpha=0.5, reg_lambda=0.1,
#                     subsample=0.7)

0.6582278481012658

In [272]:
lr_xgb_test = make_pipeline(
    preprocessor,
    SelectFromModel(
        LogisticRegression(C=0.15, penalty="l2", random_state=42, solver="saga", max_iter=10000, multi_class="ovr", 
                           n_jobs=-1, class_weight=class_weight)
    ),
    OneVsRestClassifier(
        XGBClassifier(random_state=42, verbosity=0, booster="gbtree", learning_rate=0.1, max_depth=5, reg_alpha=0.5, reg_lambda=0.1,
                     subsample=0.7)
    )
)
lr_xgb_test.fit(X_train, y_train);

c:\Users\ROG ZEPHYRUS\miniconda3\envs\capstone2023-env\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


In [273]:
lr_xgb_test.score(X_train, y_train)

1.0

In [274]:
lr_xgb_test.score(X_test, y_test)

0.5569620253164557

In [258]:
prediction_result["xgb"] = xgb_test.predict(X_test)

In [397]:
prediction_result.head(20)

,True label,pipe_lr,pipe_lr_multi,rf,lr_rf,xgb,lr_rf_ovr,lr_rf_l1
763,2,2,2,0,2,2,0,0
39,0,0,0,0,0,0,0,0
214,0,0,0,0,0,0,0,0
202,0,2,2,0,0,2,2,0
239,4,4,4,4,4,4,4,4
218,0,0,0,0,0,0,0,0
556,4,4,4,4,4,1,1,4
610,2,2,2,2,2,2,2,2
305,4,4,4,4,4,4,4,4
139,2,2,2,2,2,2,2,2


In [398]:
prediction_result.iloc[20:50]

,True label,pipe_lr,pipe_lr_multi,rf,lr_rf,xgb,lr_rf_ovr,lr_rf_l1
758,2,0,0,0,0,2,0,0
454,2,2,2,2,2,2,2,2
291,0,0,0,0,0,0,0,0
342,1,4,4,4,4,1,4,4
23,2,2,2,2,2,2,2,2
251,0,0,0,0,0,0,0,2
608,2,2,2,2,2,2,2,2
385,0,0,0,0,0,0,0,0
622,3,1,1,4,4,1,4,4
437,4,4,4,4,4,4,4,4


In [399]:
prediction_result.iloc[50:]

,True label,pipe_lr,pipe_lr_multi,rf,lr_rf,xgb,lr_rf_ovr,lr_rf_l1
630,4,4,4,4,4,4,4,4
791,0,2,2,2,2,2,0,4
658,0,0,0,4,0,4,0,0
49,4,4,4,4,4,4,4,4
78,0,0,2,0,2,2,2,0
235,2,2,2,2,2,2,2,2
441,2,2,2,2,2,2,2,2
66,4,4,1,0,2,2,0,2
265,1,2,2,2,2,1,2,2
336,0,0,0,0,0,0,0,0


In [401]:
classifiers = {
    # "lr": pipe_lr,
    # "lr multi": pipe_lr_multi,
    "rf": rf_tuned,
    # "lr rf": pipe_lr_rf,
    # "rf ovr": rf_ovr,
    "lr rf ovr": lr_rf_ovr,
    "lr rf l1": lr_rf_l1,
    # "xgb": xgb_test
}

In [402]:
averaging_model = VotingClassifier(
    list(classifiers.items()), voting="soft"
)
averaging_model.fit(X_train, y_train);

In [403]:
averaging_model.score(X_train, y_train)

0.7171145685997171

In [404]:
averaging_model.score(X_test, y_test)

0.6962025316455697

In [405]:
averaging_model_hard = VotingClassifier(
    list(classifiers.items()), voting="hard"
)
averaging_model_hard.fit(X_train, y_train);

In [406]:
averaging_model_hard.score(X_train, y_train)

0.7072135785007072

In [407]:
averaging_model_hard.score(X_test, y_test)

0.7088607594936709

In [408]:
stacking_model = StackingClassifier(list(classifiers.items()))
stacking_model.fit(X_train, y_train);

In [409]:
stacking_model.score(X_train, y_train)

0.7312588401697313

In [410]:
stacking_model.score(X_test, y_test)

0.6329113924050633